# QSAR для антимикробных пептидомиметиков: практический ноутбук

Этот ноутбук переписан в формате **учебной боевой практики**: минимум магии, максимум воспроизводимости.


## Жесткая критика исходного подхода (и почему это важно для Q1-уровня)

1. **Повторяющиеся и хаотичные ячейки** → высокий риск ошибок и невоспроизводимости.
2. **Нет строгой валидации** (иногда просто один сплит) → метрики завышаются.
3. **Смешение EDA, feature engineering и моделирования без пайплайна** → утечки данных.
4. **Нет Applicability Domain (AD)** → предсказания вне химического пространства ненадежны.
5. **Нет baseline и интерпретации ошибок** → непонятно, лучше ли модель тривиального предсказания.

Ниже — версия, которая ближе к хорошей научной практике.


In [ ]:
# Если запускаешь локально впервые, раскомментируй:
# !pip install rdkit-pypi scikit-learn pandas numpy matplotlib seaborn


In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors

from sklearn.model_selection import RepeatedKFold, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import make_scorer, mean_absolute_error, mean_squared_error, r2_score


## 1) Загрузка и базовая очистка

In [ ]:
DATA_PATH = 'potok.csv'  # ожидаются столбцы: smiles, activity

df = pd.read_csv(DATA_PATH)
print('Raw shape:', df.shape)
df.head()


In [ ]:
required_cols = {'smiles', 'activity'}
missing = required_cols - set(df.columns)
if missing:
    raise ValueError(f'В файле отсутствуют обязательные столбцы: {missing}')

df = df[['smiles', 'activity']].copy()
df['activity'] = pd.to_numeric(df['activity'], errors='coerce')
df = df.dropna(subset=['smiles', 'activity'])
df = df[df['activity'] > 0]  # для логарифма

df['mol'] = df['smiles'].apply(Chem.MolFromSmiles)
df = df[df['mol'].notnull()].copy()
df = df.drop_duplicates(subset='smiles').reset_index(drop=True)
df['logACT'] = np.log10(df['activity'])

print('Clean shape:', df.shape)
df.head()


## 2) Дескрипторы: понятная функция без ручного копипаста

In [ ]:
descriptor_names = [
    'MolWt', 'MolLogP', 'NumHDonors', 'NumHAcceptors',
    'NumHeteroatoms', 'NumRotatableBonds', 'TPSA',
    'NumAromaticRings', 'RingCount', 'FractionCSP3'
]

calc = MoleculeDescriptors.MolecularDescriptorCalculator(descriptor_names)

def calc_descriptor_frame(mols):
    rows = []
    for mol in mols:
        vals = calc.CalcDescriptors(mol)
        rows.append(vals)
    return pd.DataFrame(rows, columns=descriptor_names)

X_desc = calc_descriptor_frame(df['mol'])
y = df['logACT'].to_numpy()

print('Descriptor matrix:', X_desc.shape)
X_desc.head()


## 3) Morgan fingerprints (битовые признаки)

In [ ]:
def morgan_fp_array(mol, radius=2, n_bits=2048):
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=n_bits)
    arr = np.zeros((n_bits,), dtype=np.int8)
    DataStructs.ConvertToNumpyArray(fp, arr)
    return arr

X_fp = np.vstack([morgan_fp_array(m) for m in df['mol']])
print('Fingerprint matrix:', X_fp.shape)


## 4) Честная оценка: Repeated K-Fold + baseline

In [ ]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

scoring = {
    'MAE': make_scorer(mean_absolute_error, greater_is_better=False),
    'RMSE': make_scorer(rmse, greater_is_better=False),
    'R2': make_scorer(r2_score)
}

cv = RepeatedKFold(n_splits=5, n_repeats=10, random_state=42)

models = {
    'Ridge_descriptors': Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler()),
        ('model', Ridge(alpha=1.0))
    ]),
    'RF_descriptors': Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('model', RandomForestRegressor(
            n_estimators=500,
            random_state=42,
            min_samples_leaf=2,
            n_jobs=-1
        ))
    ]),
    'Ridge_fingerprint': Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('scaler', StandardScaler(with_mean=False)),
        ('model', Ridge(alpha=2.0))
    ])
}


In [ ]:
results = []

def summarize_cv(name, model, X, y):
    out = cross_validate(model, X, y, cv=cv, scoring=scoring, n_jobs=-1)
    return {
        'model': name,
        'MAE_mean': -out['test_MAE'].mean(),
        'MAE_std': out['test_MAE'].std(),
        'RMSE_mean': -out['test_RMSE'].mean(),
        'RMSE_std': out['test_RMSE'].std(),
        'R2_mean': out['test_R2'].mean(),
        'R2_std': out['test_R2'].std(),
    }

results.append(summarize_cv('Ridge_descriptors', models['Ridge_descriptors'], X_desc, y))
results.append(summarize_cv('RF_descriptors', models['RF_descriptors'], X_desc, y))
results.append(summarize_cv('Ridge_fingerprint', models['Ridge_fingerprint'], X_fp, y))

# baseline: среднее по train в каждом фолде (ручной расчет)
b_mae, b_rmse, b_r2 = [], [], []
for train_idx, test_idx in cv.split(X_desc):
    y_train, y_test = y[train_idx], y[test_idx]
    pred = np.full_like(y_test, y_train.mean())
    b_mae.append(mean_absolute_error(y_test, pred))
    b_rmse.append(rmse(y_test, pred))
    b_r2.append(r2_score(y_test, pred))

results.append({
    'model': 'Baseline_mean',
    'MAE_mean': np.mean(b_mae), 'MAE_std': np.std(b_mae),
    'RMSE_mean': np.mean(b_rmse), 'RMSE_std': np.std(b_rmse),
    'R2_mean': np.mean(b_r2), 'R2_std': np.std(b_r2),
})

res_df = pd.DataFrame(results).sort_values('RMSE_mean')
res_df


## 5) Applicability Domain через similarity-to-train (Tanimoto)

In [ ]:
def tanimoto_to_train(test_mol, train_mols, radius=2, n_bits=2048):
    fp_test = AllChem.GetMorganFingerprintAsBitVect(test_mol, radius, nBits=n_bits)
    train_fps = [AllChem.GetMorganFingerprintAsBitVect(m, radius, nBits=n_bits) for m in train_mols]
    sims = DataStructs.BulkTanimotoSimilarity(fp_test, train_fps)
    return max(sims) if sims else 0.0

# Демонстрация на одном CV-сплите
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)
train_idx, test_idx = next(kf.split(df))

train_mols = df.loc[train_idx, 'mol'].tolist()
test_mols = df.loc[test_idx, 'mol'].tolist()

tanimoto_max = [tanimoto_to_train(m, train_mols) for m in test_mols]

ad_demo = pd.DataFrame({
    'smiles': df.loc[test_idx, 'smiles'].values,
    'logACT': df.loc[test_idx, 'logACT'].values,
    'max_train_tanimoto': tanimoto_max,
})
ad_demo.head()


In [ ]:
plt.figure(figsize=(6,4))
sns.histplot(ad_demo['max_train_tanimoto'], bins=15)
plt.title('Applicability Domain proxy: max train Tanimoto')
plt.xlabel('Max Tanimoto to train set')
plt.grid(alpha=0.3)
plt.show()


## 6) Что делать дальше, чтобы приблизиться к публикации Q1

- Добавить **scaffold split** и отдельно report для scaffold-out теста.
- Выполнить **Y-randomization** (sanity check против случайных корреляций).
- Посчитать **confidence interval** метрик (bootstrap по фолдам).
- Сделать интерпретацию: SHAP (для RF/GBM) или importance по permutation.
- Прописать **domain of applicability policy**: напр. max Tanimoto < 0.3 => low confidence.

Если хочешь, следующим шагом я дам тебе учебный блок: "ты пишешь код сам, я проверяю построчно и критикую".
